In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [2]:
df10=pd.read_csv('inputWithRank.csv')
df10.drop(labels='Unnamed: 0',axis=1,inplace=True)

In [3]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df10['EventType']= label_encoder.fit_transform(df10['EventType'])
df10.drop(labels='matchUniqueID',axis=1,inplace=True)
df10.drop(labels=['DayNum','TeamID1','TeamID2'],axis=1,inplace=True)

In [4]:
# input data 
X_train=df10[df10['Season']<=2017][['isTeam1Home', 'isTeam2Home', 'Points1', 'Points2','ElapsedSeconds', 'EventTeam', 'EventType', 'OrdRankTeam1','OrdRankTeam2']]
Y_train=df10[df10['Season']<=2017][['Win1']]

X_test=df10[df10['Season']==2018][['isTeam1Home', 'isTeam2Home', 'Points1', 'Points2','ElapsedSeconds', 'EventTeam', 'EventType', 'OrdRankTeam1','OrdRankTeam2']]
Y_test=df10[df10['Season']==2018][['Win1']]

In [5]:

model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[9,]),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



model2.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
model2.summary()
model2.fit(X_train.to_numpy(),
          Y_train.to_numpy(),
          verbose=1,
          shuffle=True,
          epochs=20,
          batch_size=512)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 80        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6346/6346 [==============================] - 8s 1ms/step - loss: 1.0551 - accuracy: 0.7096 - auc: 0.7917: 0s - loss: 1.0904 - accuracy
Epoch 2/20
6346/6346 [==============================] - 6s 1ms/step - loss: 0.6042 - accuracy: 0.7551 - auc: 0.8445
Epoch 3/20
6346/6346 [==============================] - 7s 1ms/step - loss: 0.5371 - accuracy: 0.7686 - auc: 0.8578: 0s - loss

In [6]:
y_pred=model2.predict(X_test)
y_pred=pd.DataFrame(y_pred)
y=Y_test.copy()
y.rename(columns={'Win1':'y_true'},inplace=True)

y.insert(1,'y_pred',y_pred)
y['Error']=abs(y['y_true']-y['y_pred'])**2


y['y_pred_binary']=[1 if x>=0.5 else 0 for x in y['y_pred']]
y['Misclassification']=(y['y_pred_binary']-y['y_true'])**2  # here 1=misclassification, 0 means correct prediction
print("Mean Square Error is :",y['Error'].mean())


print("Root Mean Square Error RMSE is :",np.sqrt(y['Error'].mean()))


from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
r2 = r2_score(y['y_true'], y['y_pred'])
print('R2 value is', r2)

accuracy=accuracy_score(y['y_true'], y['y_pred_binary'])
print("Accuracy is", accuracy)

print("Misclassification Error is", y['Misclassification'].mean())

Mean Square Error is : 0.12083973172417445
Root Mean Square Error RMSE is : 0.34762009683586254
R2 value is 0.5146175137955302
Accuracy is 0.8236023307716012
Misclassification Error is 0.1763976692283988
